> pip install mediapipe


In [1]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image):
    alpha = overlay_image[:, :, 3]
    mask_image = alpha / 255
    for c in range(0,3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:,:,c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1-mask_image))
        
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture("man.mp4")

image_right_eye = cv2.imread("right_eye.png", cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread("left_eye.png", cv2.IMREAD_UNCHANGED)
image_nose = cv2.imread("nose.png", cv2.IMREAD_UNCHANGED)

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.99) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve perfo+mance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:                
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose = keypoints[2]
                
                h, w, _ = image.shape 
                right_eye = (int(right_eye.x * w)-20, int(right_eye.y * h)-400)
                left_eye = (int(left_eye.x * w)+20, int(left_eye.y * h)-400)
                nose = (int(nose.x * w), int(nose.y * h))
                
                overlay(image, *right_eye, 100, 100, cv2.resize(image_right_eye, None, fx=2, fy= 2))
                overlay(image, *left_eye, 100, 100, cv2.resize(image_left_eye, None, fx=2, fy= 2))
                overlay(image, *nose, 300, 100, cv2.resize(image_nose, None, fx=2, fy= 2))
                
            cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.25, fy=0.25))
            if cv2.waitKey(5) == 27: # esc
                break
cap.release()
cv2.destroyAllWindows()

ValueError: operands could not be broadcast together with shapes (0,200) (200,200) 